In [ ]:
%serialconnect

Closing old serial Serial<id=0x7f223f129668, open=True>(port='/dev/ttyUSB2', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Connecting to Serial /dev/ttyUSB2 baud=115200
Waited 0.0 seconds for isOpen()

 ** Serial connected **

Serial<id=0x7f223f129748, open=True>(port='/dev/ttyUSB2', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)

**[ys] <class 'serial.serialutil.SerialException'>
**[ys] Attempting to use a port that is not open

.........

In [66]:
import network, time, socket

si = network.WLAN(network.STA_IF)
si.active(True)
#assert not si.isconnected()

knownwifis = dict(k.split(b":")[:2]  for k in open("wificodes.txt", "rb"))
wifiname = max((sc  for sc in si.scan()  if sc[0] in knownwifis), key=lambda X: X[3])[0]  # X[3]=RSSI (received signal strength)
print("strongest known DoESWifi signal", wifiname)
si.connect(wifiname, knownwifis[wifiname])
del knownwifis

while not si.isconnected():
    time.sleep(0.1)
    pass

ipnumber = si.ifconfig()[0]
print("Connect through wifi to", ipnumber)


strongest known DoESWifi signal b'DoESLiverpool'
.> auth (b0)
> assoc (0)
> run (10)

.Connect through wifi to 192.168.0.29


In [6]:
assert si.isconnected()

s = socket.socket()
portnumber = 9993

s.bind(socket.getaddrinfo("0.0.0.0", portnumber)[0][-1])  # recommended to always make addresses via getaddrinfo()
s.listen(1)  # no backlog of connections allowed

print("\nPlease connect to ({}, {})".format(ipnumber, portnumber))
print("mac-address {}".format(":".join(hex(x)[-2:]  for x in si.config("mac"))))
espname = "ESP_{}".format("".join(hex(x)[-2:]  for x in si.config("mac")[-3:]))



Please connect to (192.168.0.29, 9993)
mac-address 30:ae:a4:80:78:20


In [7]:
print("Waiting for a connection on ({}, {})".format(ipnumber, portnumber))
ss, addr = s.accept()
print("connection made incoming from", addr)


Waiting for a connection on (192.168.0.29, 9993)
..........connection made incoming from ('192.168.0.215', 51600)


In [9]:
ss.settimeout(0.2)
buf = []
bufsize = 0

def execcc(buf, bufsize):
    if bufsize > 50:
        return b"We will deal big things later\r\n"
    l = (b"".join(buf).decode().strip()).splitlines()
    if len(l) != 1:
        return b"We don't eval multiple lines {}\r\n".format(len(l))
    try:
        res = str(eval(l[0]))
    except Exception as e:
        res = str(e)
    return b"The answer is: {}\r\n".format(res.encode())



In [67]:
# process incoming signals from the main jupyter syste,
k = None
while True:
    try:
        if k is None:
            k = ss.recv(100)
            print("read", str(k))
    except OSError as e:
        if e.args[0] == 110: # ETIMEDOUT
            print(":", end="")
            continue
        raise
    k = k.split(b"\x04", 1)
    buf.append(k[0])
    bufsize += len(k[0])
    if len(k) == 1:
        k = None
        continue
    k = k[1]
    ss.write(b"OK")
    ss.write(b"Thank you for sending {} characters\r\n".format(bufsize))
    ss.write(execcc(buf, bufsize))
    ss.write(b"\x04\x04>")
    buf.clear()
    bufsize = 0
    

Traceback (most recent call last):
  File "<stdin>", line 12, in <module>
  File "<stdin>", line 6, in <module>
NameError: name 'ss' is not defined


In [44]:
print(ss.readline())


b'Hello-1506948768 from PC\n'


In [57]:
ss.close()

In [8]:
print(ss.recv(100))


b'thiddng'


In [19]:
ss.write(b"OK\x04\04> ")

entering workingserialreadallentering workingserialreadallentering workingserialreadall

In [18]:
ss.write(b" ")

entering workingserialreadallentering workingserialreadallentering workingserialreadall

In [62]:
%sendtofile wificodes.txt

DoESLiverpool:------:
DoESBackup:------:
The Sensible Code Company:------:

3 lines sent done